# Contract

Notebook playground to merge/contract smaller degree nodes to create a more manageable graph dataset.

In [1]:
import networkx as nx

In [2]:
graphml_file = "../data/combined_wikilink.graphml"

G = nx.read_graphml(graphml_file)
G = G.to_directed()

### Number of Nodes and Edges

In [3]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

Number of nodes: 1075674
Number of edges: 3500000


### Contraction

Search for all smaller degree nodes (within 10% less of the average degree) and merge them with their larger parent.

In [4]:
avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

avg_degree

6.507547825828271

In [5]:
PERCENT_THRESHOLD = 0.1

degree_threshold = (1 - PERCENT_THRESHOLD) * avg_degree
low_degree_nodes = [n for n, degree in G.degree() if degree < degree_threshold]

len(low_degree_nodes)

970523

In [6]:
print(f"Approximately {100 * len(low_degree_nodes) / num_nodes:3f}% nodes are low degree")

Approximately 90.224641% nodes are low degree


In [ ]:
# Group low-degree nodes by a highest-degree parent
for node in low_degree_nodes:
    neighbors = list(G.neighbors(node))
    if len(neighbors) == 0:
        continue
        
    parent = max(neighbors, key=lambda x: G.degree(x))
    G = nx.contracted_nodes(G, parent, node, self_loops=False)

# Remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))

nx.write_graphml(G, "../data/combined_wikilink_with_contracted_deg.graphml")